In [1]:
import pandas as pd

In [2]:
import duckdb
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.simplefilter('ignore')

In [3]:
with duckdb.connect('../data/protodb.db') as conn:
    df = conn.query('select * from intel.cian').df().drop_duplicates(subset=['url'])

In [4]:

df.loc[df['floor'] == df['max_floor'],'is_max_floor'] = True
df['is_max_floor'] = df['is_max_floor'].fillna(False)
df.loc[df['text'].str.contains('\d{7,}'),'is_lot'] = True

In [5]:
df.loc[:,['Улица','Дом']]

,Улица,Дом
0,мкр. Первый Московский Город-Парк,улица Никитина
1,Мичуринский проспект,56
2,улица Ивана Бабушкина,2К1
3,Сигнальный 16 ЖК,1.1
4,Скандинавия ЖК,улица Эдварда Грига
...,...,...
415,Судостроительная улица,1
416,Псковская улица,5К1
417,1-й Балтийский переулок,3/25
418,улица Саларьевская,9


In [25]:
adresses = df.loc[:,['Улица','Дом']].fillna('').apply(', '.join,axis=1)

In [28]:
import geopy
import re
gp = geopy.Nominatim(user_agent='geo_features',timeout=1.44)

In [65]:
from geopy import distance

In [60]:
center_ = gp.geocode('Москва Красная площадь').point

In [61]:
center_

Point(55.7536283, 37.62137960067377, 0.0)

100%|██████████| 92/92 [00:46<00:00,  1.97it/s]


In [78]:
def get_geofeatures(adresses:pd.Series) -> pd.DataFrame:
    
    gp = geopy.Nominatim(user_agent='geo_features',timeout=1.44)
    result = {}

    indexes = adresses.progress_apply(gp.geocode)
    postcode = indexes.apply(lambda x: re.findall('\d{5,}',x.address)[0] if x is not None else x)
    latitude = indexes.apply(lambda x: x.latitude if x is not None else x)
    longtitude = indexes.apply(lambda x: x.longitude if x is not None else x)
    centreness = indexes.apply(lambda x: distance.distance(x.point,center_).km if x is not None else x)

    result.update({
        'postcode':postcode,
        'lat':latitude,
        'long':longtitude,
        'dist_to_center':centreness}
    )

    return pd.DataFrame(result)

In [79]:
new_df = get_geofeatures(adresses)

100%|██████████| 92/92 [00:47<00:00,  1.95it/s]


In [90]:
df.join(new_df).select_dtypes(exclude='O').iloc[:,1:].corr()

,price,m2,floor,max_floor,rubm2,is_max_floor,lat,long,dist_to_center
price,1.000000,0.861258,0.127838,0.110230,0.850199,0.206533,-0.352609,0.447963,0.411817
m2,0.861258,1.000000,0.092613,0.157501,0.542628,0.094520,-0.268642,0.237601,0.235200
floor,0.127838,0.092613,1.000000,0.563734,0.171969,0.224292,-0.122207,0.080532,0.113417
max_floor,0.110230,0.157501,0.563734,1.000000,0.131002,-0.110544,-0.011552,-0.051705,-0.045848
rubm2,0.850199,0.542628,0.171969,0.131002,1.000000,0.306925,-0.337052,0.497970,0.455476
is_max_floor,0.206533,0.094520,0.224292,-0.110544,0.306925,1.000000,0.034190,-0.047326,-0.060813
lat,-0.352609,-0.268642,-0.122207,-0.011552,-0.337052,0.034190,1.000000,-0.733325,-0.766454
long,0.447963,0.237601,0.080532,-0.051705,0.497970,-0.047326,-0.733325,1.000000,0.948030
dist_to_center,0.411817,0.235200,0.113417,-0.045848,0.455476,-0.060813,-0.766454,0.948030,1.000000


In [94]:
df['text'].iloc[0].split('.')

['В продаже 3- комнатная квартира улучшенной планировки комфорт класса с параметрами :Этаж - 6Этажей в доме 19В доме пассажирский и грузовой лифтТип дома - монолитно- кирпичный Общая Площадь квартиры - 76 кв',
 'м из них жилая - 50Просторная Кухня - 14 кв',
 'м Комнаты изолированы друг от друга - 18; 15; 15 кв',
 'метр ( комнаты изолированы)Санузел раздельный Квартира распашонка окна выходят на обе стороны Благодаря продуманной застройке этот район продолжает оставаться удобным для жильцов',
 ' Здесь достаточно пространства в шаговой доступности школа, детский сад, магазины, рынок , аптека ',
 ' Все готово для продажи квартиры вам остается только посмотреть и решить',
 ' Один взрослый собственник, обременений нет ',
 ' Звоните!']

In [34]:
def get_text_features(text_series:pd.Series):

    result = {}

    is_lot = text_series.str.contains('\d{5,}')
    is_jk = text_series.str.contains('жк')
    has_park = text_series.str.contains('\\bпарк\\b')
    wc_type = (text_series.str.extract('санузел (\w{2,})').iloc[:,0].fillna('') + 
                text_series.str.extract('(\w{2,}) санузел').iloc[:,0].fillna('')
    ).where(lambda x: x!='')

    result.update(
        {
            'is_lot':is_lot,
            'is_jk':is_jk,
            'wc_type':wc_type,
            'has_park':has_park,
        }
    )

    return pd.DataFrame(result)

In [35]:
get_text_features(df['text'].str.lower())

,is_lot,is_jk,wc_type,has_park
0,False,False,раздельный,False
1,False,True,гостевой,True
2,True,False,совмещенный,False
3,True,False,NaN,False
4,False,True,NaN,True
...,...,...,...,...
415,True,False,NaN,True
416,True,False,NaN,False
417,True,False,NaN,False
418,True,False,NaN,False


In [37]:
df.columns

Index(['datetime', 'price', 'publish_delta', 'url', 'id', 'text', 'Город',
       'Округ', 'Метро', 'Район', 'Улица', 'Дом', 'metro_branch', 'metro_name',
       'metro_dist', 'img_list', 'rooms', 'm2', 'floor', 'max_floor', 'rubm2',
       'is_max_floor', 'is_lot'],
      dtype='object')

In [128]:
df['text'].iloc[3].split('.')

['Код объекта: 717212',
 'Продается просторная 3-х комнатная квартира квартира в жилом комплексе комфорт класса от застройщика "ПИК"',
 ' Корпус введен в эксплуатацию и идет выдача ключей до 30',
 '05',
 '2023',
 ' Квартира расположена на 9-м этаже, продается в чистовой отделке от застройщика',
 ' Территория полностью обустроена, есть детские площадки для разных возрастов и места отдыха для взрослых',
 'Локация очень удобная, на севере Москвы',
 ' МЦК и станция метро "Владыкино" в шаговой доступности от дома',
 ' Так же рядом находятся автобусные остановки',
 ' Рядом ВДНХ и ботанический сад',
 ' Отличные места для прогулок на свежем воздухе',
 ' Магазины, ТЦ, поклиники, кафе и тд- рядом, можно дойти пешком либо 1-2 остановки на автобусе',
 ' 1 взрослый собственник, оперативный выход на сделку',
 'Возможно приобретение в ипотеку, до получения ключей доступна семейная ипотека',
 'Звоните']

In [40]:

base_url = 'https://dom.mingkh.ru'
page_table = pd.read_html(f'https://dom.mingkh.ru/moskva/moskva/houses?page={1}',extract_links='body')[0]

In [42]:

def parse_home_page(home_url):
    df_list = pd.read_html(home_url)
    _id = home_url.split('/')[-1]
    home_data = pd.concat(df_list,axis=0)\
                    .iloc[:,:3].dropna(thresh=1)\
                    .drop_duplicates(subset=0)\
                    .set_index(0).fillna('')\
                    .fillna(method='ffill',axis=1).iloc[:,1]\
                    .rename(_id)
    return home_data.rename_axis('id').to_frame().T

In [ ]:
parse_home_page

In [46]:
home_url = page_table['Адрес'].iloc[0][1]

In [48]:
data_list = pd.read_html(base_url+home_url)

In [114]:
pd.concat(data_list).iloc[:,:3]\
    .dropna(thresh=1)\
    .set_index(0)\
    .fillna(method='ffill',axis=1)\
    .iloc[:,1]

0
Год ввода в эксплуатацию                                                                                 1967
Дом признан аварийным                                                                                     Нет
Состояние дома                                                                                      Исправный
Количество квартир                                                                                        176
Количество нежилых помещений                                                                                6
Класс энергетической эффективности                                                                 E (Низкий)
Количество подъездов                                                                                        6
Количество лифтов                                                                                           6
Наибольшее количество этажей                                                                                9
Наименьш

In [116]:

base_data = pd.read_csv('../base_home_data.csv')